In [ ]:
import os
import re
import nltk
from nltk.stem import SnowballStemmer

def leer_archivos(carpeta):
    documentos = {}

    for archivo in os.listdir(carpeta):
        match = re.search(r"\d+", archivo)  # Extraer el numero
        doc_id = int(match.group())
        with open(os.path.join(carpeta, archivo), "r", encoding="utf-8") as f:
            documentos[doc_id] = f.read()

    return documentos

def stemming(tokens):
    stemmer = SnowballStemmer("spanish")
    tokens_procesados = [stemmer.stem(token) for token in tokens]
    return tokens_procesados

def sacar_acentos(texto):
    texto = re.sub(r"[áàäâã]", "a", texto)
    texto = re.sub(r"[éèëê]", "e", texto)
    texto = re.sub(r"[íìïî]", "i", texto)
    texto = re.sub(r"[óòöôõ]", "o", texto)
    texto = re.sub(r"[úùüû]", "u", texto)
    return texto

def procesar(texto, doc_id):
    texto = texto.lower()  # Convertir a minusculas
    texto = sacar_acentos(texto)  # Eliminar acentos
    texto = re.sub(r"[^a-z\s]", "", texto)  # Eliminar caracteres especiales
    tokens = texto.split()  # Tokenizar (En este caso separo por espacios)
    tokens = stemming(tokens)

    global nTokens
    global datos

    nTokens += len(tokens)
    for token in tokens:
        if token not in datos:
            datos[token] = {
                "docsId": [],
                "df": 0,
            }  # Inicializo la estructura de cada termino

        if doc_id not in datos[token]["docsId"]:
            datos[token]["docsId"].append(doc_id)
            datos[token]["df"] = len(datos[token]["docsId"])


def analisis_lexico(carpeta):
    global nDocumentos
    documentos = leer_archivos(carpeta)
    for doc_id, contenido in documentos.items():
        procesar(contenido, doc_id)
        nDocumentos += 1


datos = {}
nTokens = 0
nDocumentos = 0
path = "collection_test/TestCollection"

analisis_lexico(path)

- Lista de términos y su DF

In [2]:
for termino, info in datos.items():
    print(f"{termino}: {info['df']}")

gat: 400
lor: 500
telescopi: 1500
botell: 2000
automovil: 4000
alfombr: 10000
calefactor: 1200
lluvi: 3200
manual: 1300
ventanal: 1100
comput: 2200
reloj: 700
llav: 800
puert: 900
tenis: 1300
pelot: 1400
mes: 600
perr: 300
sillon: 1000
cas: 200


- Cantidad de tokens

In [3]:
print(nTokens)

793254


- Cantidad de términos 

In [4]:
nTerminos = len(list(datos.keys()))
print(nTerminos)

20


- Cantidad de documentos procesados  

In [5]:
print(nDocumentos)

10000


Tests

In [6]:
import json

with open("collection_test/collection_data.json", "r", encoding="utf-8") as f:
    test_data = json.load(f)
    for term in test_data['data']:
        termino = term['term']
        df = term['df']
        if datos[termino]['df'] != df:
            print("Fallo en el test - " + termino)
            exit
        
    if (nTokens != test_data['statistics']['num_tokens']):
        print("Fallo en el test nTokens")
        exit
        
    if (nDocumentos != test_data['statistics']['N']):
        print("Fallo en el test nDocumentos")
        exit
        
    if (nTerminos != test_data['statistics']['num_terms']):
        print("Fallo en el test nTerminos")
        exit
        
    print("Test exitoso!")
        

KeyError: 'casa'